In [1]:

from pymilvus import connections, db, FieldSchema, CollectionSchema, DataType, Collection, utility
from elasticsearch import Elasticsearch, helpers

import pandas as pd

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

/Users/riter/anaconda3/envs/cinema/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_movies = pd.read_csv('../data/merged_movies.csv')
df_movies = df_movies.dropna(how='any')
df_movies_small = df_movies.iloc[:800]

# Mulvis

In [14]:
conn = connections.connect(host="127.0.0.1", port=19530)

In [15]:
db.list_database()

['default', 'movies_database']

In [16]:
db.using_database("movies_database")

In [17]:
# Загрузка модели и токенизатора
model_name = 'intfloat/multilingual-e5-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Функция для генерации эмбеддингов
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    # Используем среднее по последнему скрытому слою
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

# Обработка каждого фильма для получения эмбеддинга
def generate_movie_embeddings(movie):
    # Конкатенация данных из нужных колонок
    text = f"{movie['title']} | {movie['genres']} | {movie['overview']} | {movie['production_countries']} | {movie['runtime']} | {movie['spoken_languages']} | {movie['vote_average']} | {movie['vote_count']}"
    return get_embedding(text)

In [18]:
# Функция для поиска фильмов по текстовому запросу
def search_movies_by_milvus(query, collection_name="movies_collection", top_k=5):
    # Получение эмбеддинга запроса
    query_embedding = get_embedding(query)
    
    # Подключение к коллекции Milvus
    collection = Collection(collection_name)
    collection.load()  # Загрузка коллекции в память (если не загружена)
    
    # Параметры поиска
    search_params = {
        "metric_type": "IP",  # Тип метрики для расстояния
        "params": {"nprobe": 10}  # Число кластеров для поиска
    }
    
    # Выполнение поиска в Milvus
    results = collection.search(
        data=[query_embedding],  # Векторный запрос
        anns_field="embedding",  # Поле для поиска
        param=search_params,
        limit=top_k,  # Число возвращаемых результатов
        output_fields=["movie_id"]  # Возвращаем только ID фильмов
    )
    
    # Форматирование результатов
    movie_ids = [result.id for result in results[0]]
    distances = [result.distance for result in results[0]]
    return list(zip(movie_ids, distances))  # Возвращаем ID фильмов и их расстояния

# Пример использования функции
query = "star wars"
top_movies = search_movies_by_milvus(query)
print("Top recommended movies:")
for movie in top_movies:
    print(movie, df_movies_small[df_movies_small['movieId'] == movie[0]]['title'])

Top recommended movies:
(260, 23.380657196044922) 253    Star Wars: Episode IV - A New Hope (1977)
Name: title, dtype: object
(316, 23.26653289794922) 308    Stargate (1994)
Name: title, dtype: object
(44, 23.04047393798828) 43    Mortal Kombat (1995)
Name: title, dtype: object
(541, 23.00408935546875) 531    Blade Runner (1982)
Name: title, dtype: object
(386, 22.997859954833984) 377    S.F.W. (1994)
Name: title, dtype: object


# ElasticSearch

In [8]:
# Подключение к Elasticsearch
es = Elasticsearch("http://localhost:9200", http_auth=("elastic", "password123"))

/var/folders/3_/0zlq__wj47xg9llgg6qg6jzc0000gn/T/ipykernel_16001/3542443314.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch("http://localhost:9200", http_auth=("elastic", "password123"))


In [9]:
def search_movie_by_elastic(query, index_name="movies", top_k=5):
    """
    Функция для поиска фильмов по текстовому запросу.
    
    Args:
        query (str): Текст запроса.
        index_name (str): Название индекса.
        top_k (int): Количество возвращаемых результатов.
    
    Returns:
        list: Список найденных фильмов.
    """
    # Тело запроса
    search_body = {
        "query": {
            "function_score": {
                "query": {
                    "multi_match": {
                        "query": query,
                        "fields": ["title^3", "genres^2", "overview", "production_countries", "spoken_languages"],
                        "fuzziness": "AUTO"
                    }
                },
                "boost_mode": "multiply",
                "functions": [
                    {
                        "field_value_factor": {
                            "field": "vote_average",
                            "factor": 1.5,
                            "missing": 0
                        }
                    },
                    {
                        "field_value_factor": {
                            "field": "vote_count",
                            "factor": 1.2,
                            "missing": 1
                        }
                    }
                ]
            }
        },
        "size": top_k
    }
    
    # Выполнение запроса
    response = es.search(index=index_name, body=search_body)
    
    # Обработка результатов
    results = []
    for hit in response["hits"]["hits"]:
        results.append({
            "movieId": hit["_source"]["movieId"],
            "title": hit["_source"]["title"],
            "genres": hit["_source"]["genres"],
            "overview": hit["_source"]["overview"],
            "score": hit["_score"]  # Релевантность к запросу
        })
    
    return results

# Пример использования функции
query = "star wars"
top_movies = search_movie_by_elastic(query)
print("Top recommended movies by elastic search:")
for movie in top_movies:
    print(f"{movie['movieId']}, {movie['title']}")


Top recommended movies by elastic search:
260, Star Wars: Episode IV - A New Hope (1977)
1196, Star Wars: Episode V - The Empire Strikes Back (1980)
1210, Star Wars: Episode VI - Return of the Jedi (1983)
2628, Star Wars: Episode I - The Phantom Menace (1999)
33493, Star Wars: Episode III - Revenge of the Sith (2005)


# Final ranking

In [11]:
reranker_model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name)

In [13]:
# Функция для финального ранжирования
def final_ranking(query, top_k=5):
    # Получение кандидатов из Milvus
    milvus_candidates = search_movies_by_milvus(query)
    milvus_results = [
        {
            "movieId": movie_id,
            "title": df_movies_small[df_movies_small['movieId'] == movie_id]['title'].values[0],
            "overview": df_movies_small[df_movies_small['movieId'] == movie_id]['overview'].values[0]
        }
        for movie_id, _ in milvus_candidates
    ]

    # Получение кандидатов из Elasticsearch
    elastic_results = search_movie_by_elastic(query)

    # Объединение уникальных кандидатов по movieId
    unique_candidates = {result["movieId"]: result for result in milvus_results + elastic_results}.values()

    # Формирование входных данных для реранкера
    inputs = []
    for candidate in unique_candidates:
        combined_text = f"{candidate['title']} | {candidate['overview']}"
        inputs.append((query, combined_text))

    # Токенизация запрос-кандидат пар
    encoded_inputs = reranker_tokenizer(
        [query for query, _ in inputs],
        [text for _, text in inputs],
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    # Вычисление оценок релевантности
    with torch.no_grad():
        outputs = reranker_model(**encoded_inputs)
        scores = outputs.logits[:, 0]

    # Добавление ранжирующих оценок к кандидатам
    reranked_candidates = [
        {**candidate, "rank_score": score.item()}
        for candidate, score in zip(unique_candidates, scores)
    ]

    # Сортировка кандидатов по оценке ранжирования и выбор top_k
    final_results = sorted(reranked_candidates, key=lambda x: x["rank_score"], reverse=True)[:top_k]

    return final_results

# Пример использования функции
query = "star wars"
top_k_results = final_ranking(query, top_k=5)

print("Top final recommended movies:")
for result in top_k_results:
    print(result["movieId"], result["title"], result["rank_score"])


Top final recommended movies:
316 Stargate (1994) 5.921955585479736
44 Mortal Kombat (1995) 5.908050537109375
541 Blade Runner (1982) 5.899562835693359
386 S.F.W. (1994) 5.888994216918945
260 Star Wars: Episode IV - A New Hope (1977) 3.187070846557617
